In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import gtsa
import pandas as pd

### Lazy load co-registered DEMs from S3

In [3]:
s3_bucket_name = 'petrichor'
folder = 'rainier/dems/'

In [11]:
cog_urls = gtsa.io.parse_urls_from_S3_bucket(s3_bucket_name,
                                             aws_server_url = 's3.us-west-2.amazonaws.com',
                                             folder = folder)

date_strings = gtsa.io.parse_timestamps(cog_urls,
                                        date_string_pattern='....-..-..')

date_times = [pd.to_datetime(x) for x in date_strings]

In [12]:
len(cog_urls) == len(date_times)

True

In [9]:
ref_dem = cog_urls[-1]
ref_dem

'http://petrichor.s3.us-west-2.amazonaws.com/rainier/dems/dem_2008-10-01_WA_reproj_COG.tif'

In [124]:
ds = gtsa.io.xr_stack_geotifs(cog_urls,
                              date_times,
                              ref_dem,
                              resampling="bilinear",
                              save_to_nc = True,
                              nc_out_dir = './nc_tmp_data_rainier',
                              overwrite = False)

Saved .nc files in nc_tmp_data_rainier


### Write to Zarr file

In [138]:
ds_zarr = gtsa.io.create_zarr_stack(ds,
                                    output_directory = './zarr_stack_rainier',
                                    overwrite = False)

zarr_stack_rainier/stack.zarr
zarr_stack_rainier/stack_tmp.zarr

Zarr file exists

Zarr file info
/
 ├── band1 (21, 13133, 21786) float32
 ├── time (21,) int32
 ├── x (21786,) float64
 └── y (13133,) float64
Name               : /band1
Type               : zarr.core.Array
Data type          : float32
Shape              : (21, 13133, 21786)
Chunk shape        : (21, 821, 1362)
Order              : C
Read-only          : False
Compressor         : Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)
Store type         : zarr.storage.DirectoryStore
No. bytes          : 24033705192 (22.4G)
No. bytes stored   : 3321208369 (3.1G)
Storage ratio      : 7.2
Chunks initialized : 256/256


Determining optimal chunk size for processing
Chunk shape: (21,120,200)
Chunk size: 2016000 (2.016G)
